<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=0c618b12fcbdb2ab1ff2afbaaaed8cc5ec5b1635630e76ddb636483351c03e26
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-01 11:17:45
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.43 C
-------------------
Today PnL: 21.65 K (0.15%)
Current PnL: -25.75 L (-16.42%)
CY Booked + Current PnL: -11.46 L (-7.31%)
-------------------
Total profit:  1.59 L
Total loss:  -27.34 L
-------------------
Total Booked + Current PnL: 15.52 L (11.95%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.97%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 90.86 L (63.41%)
Deployed:  1.30 C
Current:  1.43 C
CAGR/XIRR %: 5.46%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,-0.08,-19.87,24.91,0.09,20116.0,-20022.0,80755.0,87.41,38.0,M-SC,3.37,253.0,-1.00,0.56,6.09,OX40N,NTT,DURABLES
52,NESTLEIND,1377.00,-0.11,13.43,7.03,21.40,21165.0,35638.0,301064.0,5.18,65.0,X-LC,5.13,12.0,1.68,2.10,20.60,XY25,NTT,FMCG
83,VOLTAS,1530.00,1.03,7.58,11.26,19.69,23227.0,14538.0,206280.0,-2.87,51.0,X-MC,2.05,78.0,0.63,1.44,14.89,XY25,NTT,AC
50,MASFIN,398.61,-1.44,-2.34,24.98,22.05,23902.0,-2295.0,95685.0,-15.99,52.0,H-SC,6.24,164.0,-0.10,0.67,39.09,XR,ATH,FINANCE
40,INDIGOPNTS,1408.00,3.86,-15.62,18.53,0.02,27295.0,-27259.0,147300.0,124.89,51.0,M-SC,7.07,234.0,-1.00,1.03,28.13,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.0,6.01,-32.62,67.56,12.90,99463.0,-71274.0,147222.0,116.35,66.0,H-SC,11.88,161.0,-0.72,1.03,22.41,OX40N,NTT,FINANCE
40,INDIGOPNTS,1408.0,3.86,-15.62,18.53,0.02,27295.0,-27259.0,147300.0,124.89,51.0,M-SC,7.07,234.0,-1.00,1.03,28.13,OX40N,NTT,PAINTS
71,SYMPHONY,1306.0,2.77,-30.63,44.10,-0.03,52358.0,-52416.0,118725.0,-44.37,65.0,M-SC,15.83,206.0,-1.00,0.83,7.89,OX40N,NTT,DURABLES
9,BAJAJHFL,181.5,2.04,-17.55,88.57,55.47,177658.0,-42701.0,200585.0,-26.96,43.0,X-MC,16.46,64.0,-0.24,1.40,2.78,X40N,ATH,FINANCE
69,STARHEALTH,761.0,1.78,-10.39,64.33,47.26,162064.0,-29197.0,251926.0,22.40,48.0,H-SC,9.34,171.0,-0.18,1.76,34.98,XY24,NTT,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,DIXON,18931.72,-1.50,-20.62,58.82,26.07,91147.0,-40261.0,154960.0,-61.38,29.0,X-MC,16.62,56.0,-0.44,1.08,1.30,X40N,ATH,IT
50,MASFIN,398.61,-1.44,-2.34,24.98,22.05,23902.0,-2295.0,95685.0,-15.99,52.0,H-SC,6.24,164.0,-0.10,0.67,39.09,XR,ATH,FINANCE
79,UNITDSPR,1644.00,-1.43,4.53,15.53,20.77,38011.0,10612.0,244756.0,-4.59,49.0,X-MC,1.52,62.0,0.28,1.71,11.76,X40N,NTT,BREWERIES
53,PAGEIND,51605.07,-1.26,-11.65,45.00,28.11,80078.0,-23460.0,177950.0,-35.98,34.0,X-MC,15.34,55.0,-0.29,1.24,0.07,X40,ATH,APPARELS
2,ABBOTINDIA,35195.00,-1.24,-3.84,22.78,18.07,39179.0,-6865.0,171990.0,-17.58,49.0,X-MC,7.64,63.0,-0.18,1.20,13.97,X40,ATH,PHARMA


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,0.15,0.12,81.46,81.67,76509.0,112.0,93922.0,8461.11,67.0,L-SC,17.08,271.0,0.0,0.66,81.88,XR,NTT,CERAMICS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1641.55,0.13,-4.93,118.68,107.91,166593.0,-7272.0,140372.0,-22.58,46.0,M-SC,11.42,216.0,-0.04,0.98,4.66,OX40N,ATH,CABLES
40,INDIGOPNTS,1408.00,3.86,-15.62,18.53,0.02,27295.0,-27259.0,147300.0,124.89,51.0,M-SC,7.07,234.0,-1.00,1.03,28.13,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.08,-19.87,24.91,0.09,20116.0,-20022.0,80755.0,87.41,38.0,M-SC,3.37,253.0,-1.00,0.56,6.09,OX40N,NTT,DURABLES
48,KANSAINER,340.00,1.65,-20.12,42.06,13.47,90596.0,-54270.0,215397.0,-67.15,59.0,H-SC,6.30,158.0,-0.60,1.50,9.52,XY24,NTT,PAINTS
67,SIS,528.00,-0.30,-23.66,58.84,21.26,49874.0,-26270.0,84762.0,2000.52,51.0,H-SC,4.71,166.0,-0.53,0.59,14.52,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,0.15,0.12,81.46,81.67,76509.0,112.0,93922.0,8461.11,67.0,L-SC,17.08,271.0,0.00,0.66,81.88,XR,NTT,CERAMICS
85,WIPRO,420.0,1.17,9.24,57.69,72.27,106792.0,15665.0,185113.0,-6.08,61.0,M-LC,5.42,99.0,0.15,1.29,15.99,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.17,9.24,57.69,72.27,106792.0,15665.0,185113.0,-6.08,61.0,M-LC,5.42,99.0,0.15,1.29,15.99,XR,NTT,IT
6,ASIANTILES,137.00,0.15,0.12,81.46,81.67,76509.0,112.0,93922.0,8461.11,67.0,L-SC,17.08,271.0,0.00,0.66,81.88,XR,NTT,CERAMICS
39,INDIAMART,4810.62,-0.34,-4.77,117.09,106.72,137520.0,-5888.0,117448.0,-52.92,41.0,H-SC,6.09,139.0,-0.04,0.82,17.49,AR,ATH,MISC
26,FINCABLES,1641.55,0.13,-4.93,118.68,107.91,166593.0,-7272.0,140372.0,-22.58,46.0,M-SC,11.42,216.0,-0.04,0.98,4.66,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,-0.26,-3.83,41.04,35.63,82712.0,-8033.0,201541.0,-17.18,42.0,H-MC,2.83,119.0,-0.10,1.41,12.20,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,DIXON,18931.72,-1.50,-20.62,58.82,26.07,91147.0,-40261.0,154960.0,-61.38,29.0,X-MC,16.62,56.0,-0.44,1.08,1.30,X40N,ATH,IT
35,HONAUT,58357.33,0.37,-19.14,77.24,43.32,101725.0,-31172.0,131700.0,-30.52,30.0,X-SC,7.51,90.0,-0.31,0.92,1.12,X40N,ATH,ELECTRICAL
53,PAGEIND,51605.07,-1.26,-11.65,45.00,28.11,80078.0,-23460.0,177950.0,-35.98,34.0,X-MC,15.34,55.0,-0.29,1.24,0.07,X40,ATH,APPARELS
24,DMART,5391.45,-0.79,-9.80,43.69,29.60,98360.0,-24468.0,225132.0,-23.04,35.0,X-LC,8.86,19.0,-0.25,1.57,10.23,X40N,ATH,FMCG
3,ACC,3906.00,0.00,-26.97,124.78,64.15,216830.0,-64181.0,173770.0,-57.19,36.0,X-SC,5.81,82.0,-0.30,1.21,0.72,XY24,BTT,CEMENT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,4671.5,0.20,-17.53,52.19,25.51,80099.0,-32620.0,153475.0,-5.83,41.0,H-LC,0.90,49.0,-0.41,1.07,13.10,AR,ATH,ELECTRICAL
79,UNITDSPR,1644.0,-1.43,4.53,15.53,20.77,38011.0,10612.0,244756.0,-4.59,49.0,X-MC,1.52,62.0,0.28,1.71,11.76,X40N,NTT,BREWERIES
13,BERGEPAINT,680.0,-1.13,-5.49,28.18,21.14,60545.0,-12487.0,214852.0,-12.10,41.0,X-MC,1.83,74.0,-0.21,1.50,19.32,XY24,NTT,PAINTS
21,DABUR,735.0,-0.86,-2.39,47.22,43.70,114337.0,-5932.0,242136.0,-10.52,49.0,X-MC,1.97,72.0,-0.05,1.69,12.30,XY24,BTT,FMCG
44,ITC,452.0,0.60,-1.23,12.16,10.78,28864.0,-2951.0,237367.0,-41.15,49.0,X-LC,1.99,5.0,-0.10,1.66,3.94,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.07,-1.26,-11.65,45.00,28.11,80078.0,-23460.0,177950.0,-35.98,34.0,X-MC,15.34,55.0,-0.29,1.24,0.07,X40,ATH,APPARELS
3,ACC,3906.00,0.00,-26.97,124.78,64.15,216830.0,-64181.0,173770.0,-57.19,36.0,X-SC,5.81,82.0,-0.30,1.21,0.72,XY24,BTT,CEMENT
12,BATAINDIA,2096.00,0.38,-38.94,121.44,35.20,95408.0,-50106.0,78564.0,2.96,38.0,X-SC,17.33,92.0,-0.53,0.55,0.91,X40,NTT,FOOTWEAR
35,HONAUT,58357.33,0.37,-19.14,77.24,43.32,101725.0,-31172.0,131700.0,-30.52,30.0,X-SC,7.51,90.0,-0.31,0.92,1.12,X40N,ATH,ELECTRICAL
8,AWL,485.00,0.64,-24.42,103.08,53.50,235098.0,-73678.0,228073.0,-63.37,37.0,X-MC,7.14,58.0,-0.31,1.59,1.30,XY24,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,1.00,-30.05,104.20,42.84,47383.0,-19533.0,45473.0,-54.27,43.0,X-SC,31.14,83.0,-0.41,0.32,2.88,XY24,NTT,MISC
59,RELAXO,1176.00,-0.23,-47.75,191.49,52.31,145243.0,-69311.0,75849.0,-44.34,45.0,X-SC,7.41,91.0,-0.48,0.53,1.53,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,0.38,-38.94,121.44,35.20,95408.0,-50106.0,78564.0,2.96,38.0,X-SC,17.33,92.0,-0.53,0.55,0.91,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,0.63,-0.31,24.47,24.08,30968.0,-397.0,126553.0,-8.38,52.0,X-SC,7.62,89.0,-0.01,0.88,19.24,XY24,NTT,HEALTHCARE
35,HONAUT,58357.33,0.37,-19.14,77.24,43.32,101725.0,-31172.0,131700.0,-30.52,30.0,X-SC,7.51,90.0,-0.31,0.92,1.12,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.41,-11.48,36.36,20.71,121743.0,-43409.0,334828.0,-23.62,50.0,X-LC,2.91,1.0,-0.36,2.34,11.88,X40,ATH,IT
42,INFY,1972.00,0.77,10.82,21.14,34.24,73301.0,33855.0,346743.0,-13.85,55.0,X-LC,7.41,2.0,0.46,2.42,20.00,X40,NTT,IT
76,TMPV,600.00,0.23,-24.53,62.95,22.98,107315.0,-55421.0,170477.0,-21.86,59.0,X-LC,5.68,3.0,-0.52,1.19,7.22,XY24,NTT,AUTO
82,VBL,671.64,-0.41,-1.45,37.67,35.67,117247.0,-4594.0,311248.0,-13.11,60.0,X-LC,2.82,4.0,-0.04,2.17,12.20,X40N,ATH,FMCG
44,ITC,452.00,0.60,-1.23,12.16,10.78,28864.0,-2951.0,237367.0,-41.15,49.0,X-LC,1.99,5.0,-0.10,1.66,3.94,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1800.00,1.65,-35.14,104.89,32.89,52523.0,-27132.0,50074.0,-718.39,65.0,L-MC,6.65,259.0,-0.52,0.35,37.90,XR,NTT,BANKS
6,ASIANTILES,137.00,0.15,0.12,81.46,81.67,76509.0,112.0,93922.0,8461.11,67.0,L-SC,17.08,271.0,0.00,0.66,81.88,XR,NTT,CERAMICS
50,MASFIN,398.61,-1.44,-2.34,24.98,22.05,23902.0,-2295.0,95685.0,-15.99,52.0,H-SC,6.24,164.0,-0.10,0.67,39.09,XR,ATH,FINANCE
14,BSOFT,831.70,-0.17,-21.58,92.28,50.79,101785.0,-30353.0,110300.0,0.50,54.0,H-SC,10.83,151.0,-0.30,0.77,28.43,XR,ATH,IT
70,SURYODAY,216.00,0.53,-19.35,52.55,23.03,75894.0,-34649.0,144422.0,57.89,54.0,H-SC,6.57,167.0,-0.46,1.01,43.30,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,0.15,0.12,81.46,81.67,76509.0,112.0,93922.0,8461.11,67.0,L-SC,17.08,271.0,0.00,0.66,81.88,XR,NTT,CERAMICS
14,BSOFT,831.7,-0.17,-21.58,92.28,50.79,101785.0,-30353.0,110300.0,0.50,54.0,H-SC,10.83,151.0,-0.30,0.77,28.43,XR,ATH,IT
75,TITAGARH,1548.0,-0.49,-16.44,74.41,45.74,150577.0,-39816.0,202361.0,11.43,63.0,H-SC,4.40,174.0,-0.26,1.41,31.62,XY24,NTT,ENGINEERING
37,ICICIPRULI,790.0,0.08,10.67,18.12,30.73,39507.0,21024.0,218029.0,-15.19,70.0,X-MC,4.74,75.0,0.53,1.52,24.80,X40,ATH,INSURANCE
0,5PAISA,593.0,6.01,-32.62,67.56,12.90,99463.0,-71274.0,147222.0,116.35,66.0,H-SC,11.88,161.0,-0.72,1.03,22.41,OX40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.95
1,25,44.04
2,50,75.75


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.40
MC    30.09
LC    26.49
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.45
X40      23.84
X40N     12.59
XY25     10.45
XR        9.63
AR        8.58
OX40N     7.45
SR        0.99
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.66
X-LC    23.09
H-SC    22.76
M-SC    11.31
X-SC     7.85
H-MC     4.69
L-SC     1.48
M-MC     1.39
M-LC     1.29
H-LC     1.07
L-LC     1.04
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.02,-6.96,41.35
IT,12.50,-17.31,76.29
FINANCE,10.93,-13.83,61.60
MISC,6.98,-29.95,83.11
ELECTRICAL,5.80,-12.88,54.22
PAINTS,5.74,-12.33,29.02
INSURANCE,4.72,-0.23,34.53
PHARMA,3.98,-3.11,35.57
AUTO,2.81,-29.95,75.00


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3128477.0,21
AR,1341983.0,10
XR,1281889.0,13
X40,1052165.0,15
X40N,881411.0,9
OX40N,724990.0,10
XY25,395704.0,7
SR,279841.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3533192.0,24
M-SC,1437070.0,15
X-MC,1434972.0,16
X-LC,914387.0,13
X-SC,794097.0,8
H-MC,406621.0,3
L-SC,249628.0,3
M-LC,106792.0,1
H-LC,80099.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1229956.0      6
           AR         918577.0      5
M-SC       XY24       816170.0      6
H-SC       XR         782753.0      7
X-MC       X40        487377.0      7
           XY24       409980.0      3
X-LC       X40        381683.0      6
X-MC       X40N       349993.0      4
H-SC       OX40N      322065.0      4
X-SC       X40N       315811.0      3
M-SC       OX40N      314150.0      5
X-SC       XY24       295181.0      3
H-SC       SR         279841.0      2
X-LC       X40N       215607.0      2
H-MC       AR         215525.0      2
           XY24       191096.0      1
X-MC       XY25       187622.0      2
X-LC       XY24       186094.0      2
X-SC       X40        183105.0      2
M-SC       XR         178968.0      2
L-SC       XR         160853.0      2
X-LC       XY25       131003.0      3
M-SC       AR         127782.0      2
M-LC       XR         106792.0      1
L-SC       OX40N       88775.0      1
H-LC       AR          80099.0      1
L-MC       XR          52523.0      1
M-MC       XY25        46191.0      1
L-LC       XY25        30888.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 40.0 seconds
